<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/CycleGAN_for_Image_to_Image_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from PIL import Image

# Define Generator class
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 3 * 256 * 256),  # Assuming 256x256 RGB images
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z).view(z.size(0), 3, 256, 256)

# Define Discriminator class
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3 * 256 * 256, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.model(img.view(img.size(0), -1))

# Define data transforms and load dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(256),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset_A = datasets.ImageFolder('path/to/horse_images', transform=transform)
dataset_B = datasets.ImageFolder('path/to/zebra_images', transform=transform)
loader_A = DataLoader(dataset_A, batch_size=1, shuffle=True)
loader_B = DataLoader(dataset_B, batch_size=1, shuffle=True)

# Initialize models
G_A2B = Generator()
G_B2A = Generator()
D_A = Discriminator()
D_B = Discriminator()

# Optimizers
optimizer_G = optim.Adam(list(G_A2B.parameters()) + list(G_B2A.parameters()), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_A = optim.Adam(D_A.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_B = optim.Adam(D_B.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Loss functions
criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()

# Training loop
for epoch in range(200):
    for real_A, real_B in zip(loader_A, loader_B):
        # Convert data to tensors
        real_A = real_A[0].to(device)
        real_B = real_B[0].to(device)

        # Train Generators
        optimizer_G.zero_grad()
        fake_B = G_A2B(real_A)
        fake_A = G_B2A(real_B)
        loss_G_A2B = criterion_GAN(D_B(fake_B), torch.ones_like(D_B(fake_B)))
        loss_G_B2A = criterion_GAN(D_A(fake_A), torch.ones_like(D_A(fake_A)))
        loss_cycle_A = criterion_cycle(G_B2A(fake_B), real_A)
        loss_cycle_B = criterion_cycle(G_A2B(fake_A), real_B)
        loss_G = loss_G_A2B + loss_G_B2A + 10 * (loss_cycle_A + loss_cycle_B)
        loss_G.backward()
        optimizer_G.step()

        # Train Discriminators
        optimizer_D_A.zero_grad()
        loss_D_A = criterion_GAN(D_A(real_A), torch.ones_like(D_A(real_A))) + \
                   criterion_GAN(D_A(fake_A.detach()), torch.zeros_like(D_A(fake_A.detach())))
        loss_D_A.backward()
        optimizer_D_A.step()

        optimizer_D_B.zero_grad()
        loss_D_B = criterion_GAN(D_B(real_B), torch.ones_like(D_B(real_B))) + \
                   criterion_GAN(D_B(fake_B.detach()), torch.zeros_like(D_B(fake_B.detach())))
        loss_D_B.backward()
        optimizer_D_B.step()

    print(f'Epoch [{epoch+1}/200] Loss G: {loss_G:.4f}, Loss D_A: {loss_D_A:.4f}, Loss D_B: {loss_D_B:.4f}')